In [ ]:
# Import all the Python libraries required.
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime
import csv
import matplotlib.pyplot as plt

In [2]:
url = "https://www.atlasevhub.com/public/dmv/MN_EV_Registrations.csv"

In [ ]:
df = pd.read_csv(url)

In [ ]:
df.head()

In [5]:
df["Registration Year"] = pd.to_datetime(df["Registration Date"])

In [6]:
df["Registration Year"] = df['Registration Year'].dt.strftime("%Y")

In [ ]:
df.head()

In [8]:
grouped = df.groupby("Vehicle Make")

In [9]:
df_filtered = grouped.filter(lambda x: x["Vehicle Count"].sum() >= 100.)

In [ ]:
df_filtered

In [11]:
df_new = df_filtered[["ZIP Code", "Vehicle Make", "Vehicle Model", "Registration Year", "Vehicle Model Year"]]

In [ ]:
df_new.head()

In [13]:
df_new.to_csv("/home/jovyan/homework/hw2/data/FILTERED_MN_EV_Registrations.csv")

# Task: Plot the data using a Bar Graph

In [ ]:
df_new["Frequency"] = 1

In [15]:
df_new_grouped = df_new.groupby("Registration Year").count()

In [ ]:
df_new_grouped.head()

In [17]:
df_new_grouped_bar = df_new_grouped["Frequency"]

In [ ]:
df_new_grouped_bar

In [ ]:
df_new_grouped_bar.plot(kind = "bar", title = "Frequency of Registrations by Year: MN EV's Since 2010")

# Task:  React to the following statements:

##  Statement:  The largest number of new registrations was in 2023.

##  A:  Evidenced by the above bar chart, the largest number of new EV registrations in the state of MN occurred in 2023 with a total number of 124,237 EV registrations.

## Statement:  The number of new registrations slowed in 2019.

##  A:  Evidenced by the above bar chart, the number of new EV registrations in MN did indeed slow in 2019.  Registrations declined from 12,405 in 2018 to 5,425 in 2019.  I originally considered Covid to be the cause, however, Covid first appeared in China in December 2019 and did not appear in the U.S. until March of 2020.  Then I considered gas prices.  The drop in EV registrations could be correlated to a drop in gas prices between 2018 and 2019.  However, after doing some quick research, U.S. gas prices actually rose between 2018 and 2019 which should have created greater demand for EV.  Finally, I googled why EV sales dropped in the U.S. during 2019.  According to several articles the decrease in EV sales in 2019 was a result of limiited availability and restricted supply.  

# (30%) Use GeoPandas to analyze and map datasets

In [ ]:
!pip install geopandas mapclassify folium

In [21]:
import geopandas as gpd

In [ ]:
!wget https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/society_census_acs/shp_society_census_acs.zip

In [ ]:
!wget https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mngeo/bdry_zip_code_tabulation_areas/shp_bdry_zip_code_tabulation_areas.zip

In [ ]:
!mkdir data && unzip shp_society_census_acs.zip -d data

In [ ]:
!mkdir shp && unzip shp_bdry_zip_code_tabulation_areas.zip -d shp

In [26]:
df_geo = gpd.read_file("shp/")

In [ ]:
df_geo.head()

In [ ]:
df_geo.columns

In [ ]:
df_geo.plot()

In [30]:
df_acs = gpd.read_file("data/CensusACSZipCode.dbf")

In [ ]:
df_geo.shape

In [ ]:
df_acs.shape

In [ ]:
df_acs.head()

In [ ]:
df_geo.GEOID20

In [ ]:
df_acs.GEOID2

In [36]:
df_merged = df_geo.merge(gpd.GeoDataFrame(df_acs), left_on="GEOID20", right_on="GEOID2")

In [ ]:
df_merged.geometry_y

In [38]:
df_merged = df_merged.set_geometry("geometry_x")

In [ ]:
df_merged.plot("POPTOTAL")

# § Task: Explore variables in the data

# Answer the following questions:

## 1) What is the mean Houshold Income (MEDIANHHI) in the dataset?
## 2) How does this compare with the median HHI for the entire US in 2020? (You will need to find that yourself.)
## 3) Which ZIP code has the highest HHI?
## 4) What are the top 5 ZIP codes with the largest percent population under 18 years of age? (You will need to remember to use the total population of the ZIP as the denomenator for each ZIP.)
## 5) Which 5 ZIP codes have the highest percent of professional / graduate degrees?

---

### 1) Answer:

In [40]:
df_mhi = df_acs["MEDIANHHI"].mean()

In [ ]:
df_mhi # Mean Household Income for Dataset

### 2) Answer:

### Median household income in the U.S. for 2020 was 76,660.
### Median household income in MN for 2020 was 74,999
### Therefore, the MEDIANHHI in MN for 2020 was slightly less compared to the rest of the U.S.

### 3) Answer:

In [42]:
df_zip = df_acs[["GEOG_UNIT", "MEDIANHHI"]]

In [ ]:
df_zip.sort_values("MEDIANHHI", ascending=False)

### Zip Code 55424 has the highest Median Household Income at 225,119.

### 4) Answer:

In [44]:
df_under = df_acs[["GEOG_UNIT", "AGEUNDER18", "POPTOTAL"]]

In [ ]:
df_under.head()

In [ ]:
df_under["%_Under_18"] = 1

In [ ]:
df_under.head()

In [ ]:
df_under["%_Under_18"] = df_under[["AGEUNDER18", "POPTOTAL"]].apply(lambda d: d["AGEUNDER18"]/d["POPTOTAL"], axis=1)

In [ ]:
df_under.sort_values("%_Under_18", ascending=False)

# 55029 - 60%, 56030 - 52.9%, 56666 - 49.2%, 56210 - 46.6%, and 54842 - 43.6%

### 5) Answer:

In [50]:
df_pro = df_acs[["GEOG_UNIT", "GRADPROF", "POPTOTAL"]]

In [ ]:
df_pro.head()

In [ ]:
df_pro["%_Grad/Prof"] = df_pro[["GRADPROF", "POPTOTAL"]].apply(lambda d: d["GRADPROF"]/d["POPTOTAL"], axis=1)

In [ ]:
df_pro.sort_values("%_Grad/Prof", ascending=False).head(5)

### A correlation seems to be present between MedianHHI and %_Grad/Prof(which makes logical sense).  Zipcode 55424 has the third largest percent of adult population that completed post=bac, graduate, or professional degrees.  Zipcode 55424 also has the largest Median Household Income in MN.

---

# § Task: Plot an interactive demographics map of ZIP codes with high value homes

## Use the ACS data to show the ZIP codes with homes greater than 0.5M (500K). You will need to do the following:

### 1) Filter the data to the homes with value > 500 (in the ACS data)
### 2) Aggregate those and use VAL_DENOM as the denomenator to get a percentage
### 3) Add the calculated field to the filtered GeoDataFrame
### 4) Run the GeoDataFrame.explore() function on the dataset

---

### 1) Answer:

In [55]:
result = df_merged[(df_merged["VAL500_749"] > 0)|(df_merged["VAL750_999"] > 0)|(df_merged["VAL1MIL"] > 0)]

In [ ]:
df_merged

In [ ]:
result

In [ ]:
result["Aggregate_Value"] = result[["VAL500_749", "VAL750_999","VAL1MIL", "VAL_DENOM"]].apply(lambda d: (d["VAL500_749"] + d["VAL750_999"] + d["VAL1MIL"]) / d["VAL_DENOM"], axis=1)

In [ ]:
result.head()

In [ ]:
result.explore("Aggregate_Value", tooltip=False, popup=False)

---

# § Task: Plot demographics and EV

## You will now take your EV dataset from the first part and analyze it with the ACS data.

## You will find that GeoPandas works just like Pandas in allowing for operations on DataFrames.

## You will take the EV data and merge it with eht ACS data, and make the following plots:

### 1.) Plot (using GeoPandas plot()) MEDIANHHI using the ZCTA
### 2.) Plot (using GeoPandas plot()) HOMEOWNPCT using the ZCTA
### 3.) Create a correlation matrix of MEDIANHHI  and ev_count for all ZIP codes
### 4.) Plot an interactive plot (using explore()) of the correlation; to do this you will need to find the correlation for all ZIP codes then merge these back into the GeoDataFrame.

In [75]:
df_ev = df_new.value_counts("ZIP Code")

In [76]:
data_ev = pd.DataFrame(df_ev)

In [77]:
data_ev = data_ev.reset_index()

In [78]:
data_ev.columns = ["ZIP Code", "EV Count"]

In [ ]:
data_ev.head()

In [80]:
df_ev_merged = data_ev.merge(df_merged, left_on="ZIP Code", right_on="GEOG_UNIT")

In [81]:
df_ev_merged = df_ev_merged.set_geometry("geometry_x")

In [ ]:
df_ev_merged.head()

In [ ]:
df_ev_merged.plot("HOMEOWNPCT")

In [ ]:
df_ev_merged.plot("MEDIANHHI")

In [ ]:
df_ev_merged.plot("EV Count")

In [86]:
prep = df_ev_merged[["MEDIANHHI", "EV Count"]]

In [ ]:
prep.head()

In [88]:
corrM = prep.corr()

In [ ]:
corrM

In [90]:
prep_zip = df_ev_merged[["ZIP Code", "MEDIANHHI", "EV Count"]]

In [91]:
corrP = prep_zip.corr()

In [ ]:
corrP

In [94]:
df_select = df_merged[["HIGHSCHOOL", "SOMECOLLEG", "ASSOCIATE", "BACHELORS", "GRADPROF", "R300_399", "R400_499", "R500_599",  "R600_699", "R700_799", "R800_899", "R900_999", "R1000_1249", "R1250_1499", "R1500_1999", "R2000UP",  "VAL40_69", "VAL70_99", "VAL100_124", "VAL125_149", "VAL150_174", "VAL175_199", "VAL200_249",  'VAL250_299', 'VAL300_399', 'VAL400_499', 'VAL500_749', 'VAL750_999', 'VAL1MIL', 'MEDIANHHI',  'AGEUNDER18', 'AGE18_39', 'AGE40_64', 'AGE65UP', 'LIVEDALONE', 'MARRKIDS', 'UNMARRKIDS', 'FAMNOKIDS',  'NONFAMILY', 'POPTOTAL']]

In [ ]:
df_select.head()

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=5).fit(df_select.iloc[:,1:3])
gm.get_params()

In [ ]:
centers = gm.means_
print (centers)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_select.iloc[:,1:5], df_select.iloc[:,1:5], label="data")
plt.scatter(centers[:,0], centers[:,1],c="r", label="centers")
plt.legend()
plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=9).fit(df_select.iloc[:,1:3])
gm.get_params()

In [ ]:
centers = gm.means_
print (centers)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_select.iloc[:,1:5], df_select.iloc[:,1:5], label="data")
plt.scatter(centers[:,0], centers[:,1],c="r", label="centers")
plt.legend()
plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=12).fit(df_select.iloc[:,1:3])
gm.get_params()

In [ ]:
centers = gm.means_
print (centers)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(df_select.iloc[:,1:5], df_select.iloc[:,1:5], label="data")
plt.scatter(centers[:,0], centers[:,1],c="r", label="centers")
plt.legend()
plt.show()

# § Task: Use the GMM algorithm to cluster the high variance features

## 1.)Use VarianceThreshold on the entire dataset to eleminate features. set threshold=0.4
## 2.)Perform GMM as before, this time with just n_components=5
## 3.)Make an interactive plot as before, compare this plot with the previous with a 2-3 sentence summary of the differences.

In [ ]:
from sklearn.feature_selection import VarianceThreshold
thresholder = VarianceThreshold(threshold=.4)
x_high_variance = thresholder.fit_transform(df_select)
print(x_high_variance[0:7])

In [ ]:
ngm = GaussianMixture(n_components=5).fit(x_high_variance)
ngm.get_params()

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(x_high_variance[:,1:5], x_high_variance[:,1:5], label="data")
plt.legend()
plt.show()

In [247]:
df_move = df_new[["ZIP Code", "Frequency"]]

In [248]:
df_moving = df_move.groupby("ZIP Code").count()

In [ ]:
df_moving.head()

In [250]:
df_moving = pd.DataFrame(df_moving)

In [251]:
df_moving = df_moving.reset_index()

In [252]:
df_moving.columns = ["ZIP Code", "EV Count"]

In [ ]:
df_moving.head()

In [254]:
df_moving_merged = df_moving.merge(df_merged, left_on="ZIP Code", right_on="GEOG_UNIT")

In [255]:
df_moved_merged = df_moving_merged.set_geometry("geometry_x")

In [ ]:
df_moved_merged.head()

In [257]:
da_least = df_moved_merged[(df_moved_merged["EV Count"] <= 200)]

In [261]:
da_next = df_moved_merged[(df_moved_merged["EV Count"] >= 201) & (df_moved_merged["EV Count"] <= 1000)]

In [262]:
da_major = df_moved_merged[(df_moved_merged["EV Count"] >= 1001) & (df_moved_merged["EV Count"] <= 2000)]

In [263]:
da_max = df_moved_merged[(df_moved_merged["EV Count"] >= 2000)]

In [ ]:
da_least.plot("EV Count")

In [ ]:
da_next.plot("EV Count")

In [ ]:
da_major.plot("EV Count")

In [ ]:
da_max.plot("EV Count")